In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### FIX ME #####
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "HoustonLux21"
host = "nv-desktop-services.apporto.com"
port = 31448
database = "AAC"
collection = "animals"

db = AnimalShelter(username, password, host, port, database, collection)

df = pd.DataFrame.from_records(db.read({}))

# Remove '_id' column to prevent errors
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    html.Div(className='buttonRow', style={'display': 'flex'}, children=[
        html.Button(id='submit-button-one', n_clicks=0, children='Cats'),
        html.Button(id='submit-button-two', n_clicks=0, children='Dogs')
    ]),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],  # Ensures an initial selection
        page_action="native",
        page_current=0,
        page_size=10
    ),
    
    html.Br(),
    html.Hr(),

    # Placeholder for the geolocation chart
    html.Div(id='map-id', className='col s12 m6')
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks')]
)
def on_click(button1, button2):
    df = pd.DataFrame.from_records(db.read({}))
    
    if int(button1) > int(button2):
        df = pd.DataFrame.from_records(db.read({"animal_type": "Cat"}))
    elif int(button2) > int(button1):
        df = pd.DataFrame.from_records(db.read({"animal_type": "Dog"}))
    
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')


@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'data'),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    
    # Ensure a row is selected
    if selected_rows is None or len(selected_rows) == 0:
        row = 0
    else:
        row = selected_rows[0]

    # Default coordinates if missing
    lat, lon = 30.75, -97.48  # Austin TX
    if 13 in dff.columns and 14 in dff.columns:
        lat = dff.iloc[row, 13] if pd.notna(dff.iloc[row, 13]) else lat
        lon = dff.iloc[row, 14] if pd.notna(dff.iloc[row, 14]) else lon

    return dl.Map(style={'width': '1000px', 'height': '500px'},
                  center=[lat, lon], zoom=10, children=[
                      dl.TileLayer(id="base-layer-id"),
                      dl.Marker(position=[lat, lon], children=[
                          dl.Tooltip(dff.iloc[row, 4]),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row, 9])
                          ])
                      ])
                  ])


app.run_server(debug=True)

MongoDB connection successful.
Dash app running on http://127.0.0.1:14383/
